# See a Summary of All Activities

In [2]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager, GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, StepsActivities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none

from jupyter_funcs import format_number

gc_config = GarminConnectConfigManager()
db_params_dict = ConfigManager.get_db_params()


garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

def __report_sport(sport_col, sport):
    records = Activities.row_count(garmin_act_db, sport_col, sport)
    if records > 0:
        sport_title = sport.title().replace('_', ' ')
        total_distance = Activities.get_col_sum_for_value(garmin_act_db, Activities.distance, sport_col, sport)
        if total_distance is None:
            total_distance = 0
            average_distance = 0
        else:
            average_distance = total_distance / records
        return [sport_title, records, format_number(total_distance, 1), format_number(average_distance, 1)]

doc = snakemd.new_doc()

doc.add_heading("Activities Report")
doc.add_paragraph("Analysis of all activities in the database.")

doc.add_table(
    ['Type', 'Count'],
    [
        ["Total activities", Activities.row_count(garmin_act_db)],
        ["Total Lap records", ActivityLaps.row_count(garmin_act_db)],
        ["Activity records", ActivityRecords.row_count(garmin_act_db)],
        ["Fitness activities", Activities.row_count(garmin_act_db, Activities.type, 'fitness')],
        ["Recreation activities", Activities.row_count(garmin_act_db, Activities.type, 'recreation')]
    ])

years = Activities.get_years(garmin_act_db)
years.sort()
doc.add_paragraph(f"Years with activities: {len(years)}: {years}")
sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sport))
doc.add_paragraph(f"Sports: {', '.join(sports)}")
sub_sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sub_sport))
doc.add_paragraph(f"SubSports: {', '.join(sub_sports)}")

sports_stats = []
for sport_name in [sport.name for sport in fitfile.Sport]:
    sport_stat = __report_sport(Activities.sport, sport_name)
    if sport_stat:
        sports_stats.append(sport_stat)
doc.add_heading("Sport Type Statistics", 3)
doc.add_table(['Sport', 'Total Activities', f'Total Distance ({distance_units})', f"Average Distance ({distance_units})"], sports_stats)

def __format_activity(activity):
    if activity:
        if activity.is_steps_activity():
            steps_activity = StepsActivities.get(garmin_act_db, activity.activity_id)
            return [activity.activity_id, activity.name, activity.type, activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1), steps_activity.avg_pace, format_number(activity.calories)]
        return [activity.activity_id, activity.name, activity.type, activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1), '', format_number(activity.calories)]
    return ['', '', '', '', '', '', '', '', '']

activities = Activities.get_latest(garmin_act_db, 10)
rows = [__format_activity(activity) for activity in activities]
doc.add_heading("Last Ten Activities", 3)
doc.add_table(['Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories'], rows)

rows = []
for display_activity in gc_config.display_activities():
    name = display_activity.activity_name().capitalize()
    rows.append([f'Latest {name}'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Fastest {name}'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Slowest {name}'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Longest {name}'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, display_activity)))

doc.add_heading("Interesting Activities", 3)
doc.add_table(['What', 'Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories'], rows)

doc.add_heading("Courses", 3)
courses = Activities.get_col_distinct(garmin_act_db, Activities.course_id)
doc.add_paragraph(str(courses))

display(Markdown(str(doc)))

# Activities Report

Analysis of all activities in the database.

| Type                  | Count   |
| --------------------- | ------- |
| Total activities      | 896     |
| Total Lap records     | 13695   |
| Activity records      | 2824520 |
| Fitness activities    | 6       |
| Recreation activities | 245     |

Years with activities: 7: [2018, 2019, 2020, 2021, 2022, 2023, 2024]

Sports: cycling, walking, fitness_equipment, swimming, training, hiking, running, generic

SubSports: virtual_activity, generic, strength_training, lap_swimming, road, gravel_cycling, breathing, commuting, treadmill, mountain, yoga, indoor_rowing, indoor_cycling, open_water, cardio_training, pilates

### Sport Type Statistics

| Sport             | Total Activities | Total Distance (kilometers) | Average Distance (kilometers) |
| ----------------- | ---------------- | --------------------------- | ----------------------------- |
| Generic           | 1                | 0.0                         | 0.0                           |
| Running           | 97               | 678.4                       | 7.0                           |
| Cycling           | 402              | 9709.2                      | 24.2                          |
| Fitness Equipment | 8                | 0.1                         | 0.0                           |
| Swimming          | 215              | 381.7                       | 1.8                           |
| Training          | 44               | 0                           | 0                             |
| Walking           | 117              | 537.7                       | 4.6                           |
| Hiking            | 12               | 143.7                       | 12.0                          |

### Last Ten Activities

| Id          | Name                                             | Type       | Sport             | Distance (kilometers) | Elapsed Time    | Speed (kph) | Pace (per kilometers) | Calories |
| ----------- | ------------------------------------------------ | ---------- | ----------------- | --------------------- | --------------- | ----------- | --------------------- | -------- |
| 13881653578 | ROUVY - On And Off                               | training   | cycling           | 12.7                  | 00:54:00.040000 | 14.1        |                       | 545      |
| 13845399877 | ROUVY - High Cadence                             | training   | cycling           | 15.7                  | 00:40:00.046000 | 23.6        |                       | 385      |
| 13822956313 | ROUVY - Killer Hill Sprints                      | training   | cycling           | 6.5                   | 00:42:00.014000 | 9.2         |                       | 409      |
| 13791243823 | ROUVY - Fat for fuel                             | training   | cycling           | 18.2                  | 00:50:00.176000 | 21.8        |                       | 476      |
| 13784308121 | ROUVY - Forty On, Twenty Off                     | training   | cycling           | 10.8                  | 00:50:00.169000 | 13.0        |                       | 516      |
| 13684380480 | Dormagen Radfahren                               | recreation | cycling           | 3.5                   | 01:24:58.872000 | 20.3        |                       | 132      |
| 13682716477 | Krafttrai​ning                                   | training   | fitness_equipment | 0.0                   | 00:29:17.118000 | 0.0         |                       | 215      |
| 13655626882 | Dormagen Radfahren                               | recreation | cycling           | 4.3                   | 01:10:58.248000 | 17.8        |                       | 155      |
| 13640683075 | ROUVY - Bodmin Moor | Babble Ride Across Britain | training   | cycling           | 25.7                  | 01:04:11.700000 | 24.1        |                       | 710      |
| 13604812398 | ROUVY - Maximise Your Fat Burning                | training   | cycling           | 28.7                  | 01:10:00.046000 | 24.6        |                       | 678      |

### Interesting Activities

| What         | Id          | Name                                   | Type          | Sport   | Distance (kilometers) | Elapsed Time    | Speed (kph) | Pace (per kilometers) | Calories |
| ------------ | ----------- | -------------------------------------- | ------------- | ------- | --------------------- | --------------- | ----------- | --------------------- | -------- |
| Latest Walk  | 13545802042 | Dormagen Gehen                         | recreation    | walking | 2.7                   | 00:58:18.897000 | 2.7         | 00:35:09.232223       | 404      |
| Fastest Walk | 6211828691  | Dormagen Regenrunde Gehen              | recreation    | walking | 8.2                   | 01:18:03.463000 | 6.3         | 00:15:17.528083       | 497      |
| Slowest Walk | 5201104269  | Hellenthal Wildfreigehege              | recreation    | walking | 3.6                   | 03:21:37.888000 | 1.1         | 01:30:18.667539       | 928      |
| Longest Walk | 10841887931 | Paris Gehen                            | uncategorized | walking | 22.6                  | 06:23:47.639000 | 3.5         | 00:27:17.176238       | 1870     |
| Latest Run   | 9156040209  | Um die 10 mit Nancy                    | special_event | running | 10.9                  | 01:18:16.054000 | 8.4         | 00:11:33.085354       | 1208     |
| Fastest Run  | 3642630740  | Dormagen Laufen                        | training      | running | 11.0                  | 01:02:04.838000 | 10.6        | 00:05:38.409475       | 974      |
| Slowest Run  | 9144054073  | Laufband Training in der Schwitzehölle | training      | running | 0.8                   | 01:43:08.491000 | 6.0         | 00:16:06.688849       | 1439     |
| Longest Run  | 7388149928  | Dormagen - Locker laufen               | training      | running | 16.7                  | 01:47:09.110000 | 9.3         | 00:10:19.693601       | 1691     |
| Latest Ride  | 13881653578 | ROUVY - On And Off                     | training      | cycling | 12.7                  | 00:54:00.040000 | 14.1        |                       | 545      |
| Fastest Ride | 7668483570  | Indoor Cycling                         | uncategorized | cycling | 31.0                  | 00:54:26.600000 | 34.2        |                       | 552      |
| Slowest Ride | 8143957917  | Dormagen Radfahren                     | uncategorized | cycling | 2.6                   | 00:32:40.714000 | 4.8         |                       | 253      |
| Longest Ride | 11960938757 | Bubatz Runde                           | training      | cycling | 166.9                 | 10:08:12.970000 | 23.8        |                       | 4488     |

### Courses

[None, 1107755356, 111576167, 879330352, 1218197410, 805619543, 1222639772, 805620563, 1231382201, 1244144548, 1259283526, 1267158703, 1268490254, 1290567309, 1299244410, 1311125365, 1320557207, 22880913, 186557205, 188485962, 190562792, 254017576, 67193891, 453302519, 473932900, 763101427, 70608815, 104596083, 105294232, 106829420, 106258725, 108382895, 109302158, 777387667, 110771013, 63848782, 854428961, 856060214, 73437566, 913905980]